#Inicialização do navegador

In [19]:
#Criar um navegador
#Observação: utilizar Edge ou Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager 
import time

from selenium.webdriver.edge.service import Service
servico = Service(EdgeChromiumDriverManager().install())
navegador = webdriver.Edge(service=servico)



#importar base de dados
import pandas as pd
tabela_produtos = pd.read_excel('buscas.xlsx')

display(tabela_produtos)


,Nome,Termos Banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,2500,4500
1,rtx 2060,galax,1800,3500


#Definição de função do Google Shopping e Buscapé

In [20]:
def verificar_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos       

def verificar_termos_nome_produto(lista_termos_nome_produto, nome):
    tem_termos_nome_produto = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_termos_nome_produto = False
    return tem_termos_nome_produto

def busca_google_shopping(navegador,produto, termos_banidos, preco_minimo, preco_maximo):
    #tratar valores
    produto = produto.lower() #garantir que os dados da tabela fiquem minusculos
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")#transforma a string de termos banidos em lista
    lista_termos_nome_produto = produto.split(" ")#transforma a string de produtos em lista
    lista_ofertas = [] # lista vazia para armazenar os resultados da busca
    #transformar os dados do excel em float
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #entrar no google
    navegador.get('https://www.google.com/')
    
    #busca o elemento pelo xpath e envia o produto
    navegador.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto)
    navegador.find_element('xpath', '//*[@id="APjFqb"]').send_keys(Keys.ENTER)
    #entrar no google shopping com condicional
    elementos2 = navegador.find_elements('class name', 'hdtb-mitem') #versao mais antiga
    elementos = navegador.find_elements('class name', 'GKS7s') #versao mais nova
    for item in elementos:
        if "Shopping" in item.text:
            print("v1 - "+item.text)
            item.click()
            break
        
    for item2 in elementos2:
        if "Shopping" in item2.text:
            print("v2 - "+item2.text)
            item2.click()
            break 
            

    
    #Pegar card dos produtos google shopping 
    lista_resultados = navegador.find_elements('class name', 'i0X6df')
    
    #a busca ocorre apenas dentro da classe i0X6df e naõ em no navegador inteiro
    
    for resultado in lista_resultados:
        
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        #analisar se o nome não tem nenhum termo banido
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)
                
        #analisar se tem todos os termos buscados no nome do produto
        tem_termos_nome_produto = verificar_termos_nome_produto(lista_termos_nome_produto, nome)
        
        #Tratamento preco
        if not tem_termos_banidos and tem_termos_nome_produto:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("+impostos", " ")
            #remover o R$ ; remover o espaço; remover o ponto; trocar a virgula por ponto para reconhecer no float
            preco = float(preco)
            
            
            
            
            #Verificar se o preço está entre o preço minimo e maximo
            if preco_minimo <= preco <= preco_maximo:
                #Caso retornar None é necessário utilizar uma referencia
                elemento_referencia = resultado.find_element('class name', 'bONr3b') #Elemento filho da classe do Link 
                elemento_pai = elemento_referencia.find_element('xpath', ('..'))#Elemento pai do elemento filho (..) serve para voltar um nível
                link = elemento_pai.get_attribute('href') #retornando None em vez do link
                lista_ofertas.append((nome,preco,link))

    return lista_ofertas

def busca_buscape(navegador,produto, termos_banidos, preco_minimo, preco_maximo):
    
    #trata valores
    produto = produto.lower() #garantir que os dados da tabela fiquem minusculos
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")#transforma a string de termos banidos em lista
    lista_termos_nome_produto = produto.split(" ")#transforma a string de produtos em lista
    lista_ofertas = [] # lista vazia para armazenar os resultados da busca
    preco_minimo = float(preco_minimo)#converte dados do excel em float
    preco_maximo = float(preco_maximo)
    #entrar no buscape
    navegador.get('https://www.buscape.com.br/')

    
    #buscar o elemento pelo xpath e envia o produto
    navegador.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto)
    navegador.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
    
    #pegar os resultados do card do produto
    lista_resultados = navegador.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')
    
    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        link = resultado.get_attribute('href')
        print(nome, preco, link)
    
    #analisar se o nome não tem nenhum termo banido
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)
    
    #analisar se tem todos os termos buscados no nome do produto
        tem_termos_nome_produto = verificar_termos_nome_produto(lista_termos_nome_produto, nome)
    
    #analisar range de preço determidado
        if not tem_termos_banidos and tem_termos_nome_produto:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            #remover o R$ ; remover o espaço; remover o ponto; trocar a virgula por ponto para reconhecer no float
            preco = float(preco)
            
            #Verificar se o preço está entre o preço minimo e maximo
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome,preco,link))
    
    
    #retorna lista de ofertas buscape
    
    return lista_ofertas


#Construindo tabela de ofertas

In [21]:
tabela_ofertas = pd.DataFrame()#tabela para receber as ofertas

#dados da tabela
for linha in tabela_produtos.index:
    
    produto  = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos Banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(navegador,produto, termos_banidos, preco_minimo, preco_maximo)
    #verifica se está vazio
    if lista_ofertas_google_shopping:
        #transformar em dataframe
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Nome', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None #para sempre definir a tabela para na mescla não misturar produtos
    
    time.sleep(5)
    lista_ofertas_buscape = busca_buscape(navegador,produto, termos_banidos, preco_minimo, preco_maximo)
    #verifica se está vazio
    if lista_ofertas_buscape:
        #transformar em dataframe
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Nome', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
    
 
display(tabela_ofertas)


v2 - Shopping
smartphone apple iphone 12 64gb câmera dupla R$ 2.994,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb
smartphone apple iphone 12 vermelho 64gb câmera dupla R$ 3.099,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-vermelho-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb
smartphone apple iphone 12 usado 64gb câmera dupla R$ 2.469,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-usado-64gb-camera-dupla?_lc=88&searchterm=iphone%2012%2064%20gb
smartphone apple iphone 12 mini usado 64gb câmera dupla R$ 2.799,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-mini-usado-64gb-camera-dupla?_lc=88&searchterm=iphone%2012%2064%20gb
smartphone apple iphone 12 mini 64gb câmera dupla R$ 5.639,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-mini-64gb-ios-camera-dupla?_lc=88&searchterm=iphone%2012%2064%20gb
v2 - Shopping
placa de video nvidia geforce rtx 2060 

,Nome,Preço,Link
0,smartphone apple iphone 12 64gb câmera dupla,2994.00,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone 12 vermelho 64gb câmer...,3099.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de video nvidia geforce rtx 2060 6 gb gd...,2870.00,https://www.buscape.com.br/placa-de-video/plac...
1,placa de video nvidia geforce rtx 2060 gddr6 6...,2811.67,https://www.buscape.com.br/lead?oid=833745835&...
2,placa de video gigabyte geforce rtx 2060 12gb ...,2843.64,https://www.buscape.com.br/lead?oid=869745149&...
3,evga placa de vídeo nvidia geforce rtx 2060 6g...,3167.90,https://www.buscape.com.br/lead?oid=1139015123...


#Exportando tabela de ofertas para excel

In [22]:
#exportar tabela de ofertas para excel
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

#Enviando por email GMail

In [23]:
#enviar pelo gmail
import smtplib
import email.message

if len(tabela_ofertas) > 0:
    
    corpo_email = f'''
    <p>Segue abaixo as ofertas encontradas na faixa de preço desejada:</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att,Rapha TCC</p>
    '''
    msg = email.message.Message()
    msg['Subject'] = 'Ofertas encontradas na faixa de preço desejada'
    msg['From'] = 'raphaelvictor40@gmail.com'
    msg['To'] = 'raphaelvictor40@gmail.com'
    password = "ubthvlcuzixmrvba"
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email )
    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')
    

Email enviado
